In [2]:
import cv2
import sys
sys.path.insert(0, '/home/caron/Bureau/Model/HexagonalScale/ST-GNN-for-wildifre-prediction/Prediction')

from tools import *
from config import *

def calculate_iou(image1, image2):
    
    # Vérifier que les deux images sont de la même taille
    if image1.shape != image2.shape:
        raise ValueError("Les deux images doivent être de la même taille pour calculer l'IoU.")
    
    # Binariser les images (seuiling pour les valeurs de pixels 0 ou 255)
    _, binary_image1 = cv2.threshold(image1, 127, 255, cv2.THRESH_BINARY)
    _, binary_image2 = cv2.threshold(image2, 127, 255, cv2.THRESH_BINARY)
    
    # Calculer l'intersection et l'union
    intersection = np.logical_and(binary_image1, binary_image2).sum()
    union = np.logical_or(binary_image1, binary_image2).sum()
    
    # Calculer l'IoU
    iou = intersection / union if union != 0 else 0
    return iou

/home/caron/Bureau/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Nom de l'hôte : caron-Precision-7780
Adresse IP locale : 127.0.1.1


/home/caron/Bureau/.conda/lib/python3.9/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Nom de l'hôte : caron-Precision-7780
Adresse IP locale : 127.0.1.1


In [ ]:
dept = 'departement-69-rhone'
scale = 4
base = 'risk-size-watershed'
path_to_predict = Path('firemen2/firepoint/2x2/train/features_geometry/'+dept)
path_to_gt = Path('firemen/firepoint/2x2/train/features_geometry/'+dept)

image_one = read_object(f'pred_merge.pkl', path_to_predict)
image_two = read_object(f'pred_merge.pkl', path_to_gt)

calculate_iou(image_one, image_two)